# 数据选取

以前说过多个文档（多个评论/文章）提取主题，那是属于聚类的方法

这次是针对一篇文章提取关键词（类似写摘要（总结）的感觉）

选取的文章是《网约车司机二三事》https://www.jianshu.com/p/6941a40df96c

注意：如果是直接从网上复制的文本，在保存和使用的很可能出现错误

这是因为不同于英语，汉字有编码问题。不同系统都有不同的默认编码，不同版本的Python接受的编码也不同。你从网上下载的文本文件，也可能与你系统的编码不统一。

![avatar](https://upload-images.jianshu.io/upload_images/64542-7510c535903f92d0.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/700)

所以，避免掉坑的正确使用中文文本数据方式，是在Jupter Notebook里，新建一个文本文件，把**文本以任意一种能正常显示的编辑器打开**,复制到这里面,然后拷贝全部内容，粘贴即可

# 提取

## 分词

In [1]:
from jieba.analyse import *

In [3]:
with open('sample.txt',encoding='utf8') as f:
    data=f.read()

In [5]:
data[:10]

'网约车司机二三事\n\n'

数据读取正确

## 使用TF-IDF提取关键字

使用TF-idf方式提取关键词和权重，并且依次显示出来。如果你不做特殊指定的话，默认显示数量为20个关键词。

In [6]:
for keyword,weight in extract_tags(data,withWeight=True):
    print("%s %s"%(keyword,weight))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\shanshan\AppData\Local\Temp\jieba.cache
Loading model cost 1.081 seconds.
Prefix dict has been built succesfully.


优步 0.28087559478179147
司机 0.11995194759665198
乘客 0.10548612948502202
师傅 0.09588881078154185
张师傅 0.08381623349632894
目的地 0.07536185128863436
网约车 0.07021889869544787
姐姐 0.06834121277656388
自己 0.06725331106610866
上车 0.06232769163083701
活儿 0.06001343542136564
天津 0.05691580567920705
10 0.0526641740215859
开优步 0.0526641740215859
事儿 0.04855445676703377
李师傅 0.04850355019427313
天津人 0.04826536860264317
绕路 0.047824472309691626
出租车 0.04484802607483113
时候 0.044084029859148305


其实就是调用jieba分词里的分析工具，进行了TF—IDF的提取分析

观察下类别，其实关键词提取还是比较靠谱的。当然，其中也混入了个数字10，好在无伤大雅。

你需要修改关键词数量，就需要指定topK参数

In [8]:
for keyword,weight in extract_tags(data,topK=10,withWeight=True):
    print("%s %s"%(keyword,weight))

优步 0.28087559478179147
司机 0.11995194759665198
乘客 0.10548612948502202
师傅 0.09588881078154185
张师傅 0.08381623349632894
目的地 0.07536185128863436
网约车 0.07021889869544787
姐姐 0.06834121277656388
自己 0.06725331106610866
上车 0.06232769163083701


## TextRank提取关键字

In [10]:
for keyword,weight in textrank(data,topK=20,withWeight=True):
    print("%s %s"%(keyword,weight))

优步 1.0
司机 0.7494059966484155
乘客 0.5942845064572533
姐姐 0.48545874199064
天津 0.4511134903660482
目的地 0.4294100274662929
时候 0.41808386330346065
作者 0.41690383815327287
没有 0.3577645150520248
活儿 0.2913715664937725
上车 0.2770100138843037
绕路 0.27460859208431115
转载 0.2719329031862934
出来 0.24258074539320906
出租 0.2386398899911447
事儿 0.22870032271337745
单数 0.21345068036612438
出租车 0.2120496654807952
拉门 0.205816713636715
跟着 0.205134709860173


## 分析

很明显可以发现，TF-IDF的提取结果和TextRank的提取结果是不一样的，起码在TextRank中，没有数字10这个关键字

这是不是意味着TextRank方法一定优于TF-IDF呢？

不一定

### 原理

#### TF-IDF

+ 全称是：  Term Frequency - inverse document frequency
+ 中间有个连字符，左右两侧各是一部分，共同结合起来，决定某个词的重要程度。
+ 第一部分是词频（Term Frequency），一般认为某个词语出现的越多，可能会越重要，但是这只是可能，不是绝对。所以需要第二部分IDF的配合
+ 逆文档频率（inverse document frequency）首先计算某个词在各文档中出现的频率。假设一共有10篇文档，其中某个词A在其中10篇文章中都出先过，另一个词B只在其中3篇文中出现。请问哪一个词更关键？
  答案是B更关键。A可能就是虚词，或者全部文档共享的主题词。而B只在3篇文档中出现，因此很有可能是个关键词。

+ 逆文档频率就是把这种文档频率取倒数。这样第一部分和第二部分都是越高越好。二者都高，就很有可能是关键词了。

#### TextRank

相对于TF-idf，TextRank要显得更加复杂一些。它不是简单做加减乘除运算，而是基于图的计算。

![avatar](https://upload-images.jianshu.io/upload_images/64542-0642fb0d702325e3.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/700)

TextRank首先会提取词汇，形成节点；然后依据词汇的关联，建立链接。

依照连接节点的多少，给每个节点赋予一个初始的权重数值。

然后就开始迭代。

根据某个词所连接所有词汇的权重，重新计算该词汇的权重，然后把重新计算的权重传递下去。直到这种变化达到均衡态，权重数值不再发生改变。这与Google的网页排名算法PageRank，在思想上是一致的。

根据最后的权重值，取其中排列靠前的词汇，作为关键词提取结果。

https://blog.csdn.net/u013041398/article/details/52473994
###  使用TextRank提取关键字
关键词抽取的任务就是从一段给定的文本中自动抽取出若干有意义的词语或词组。将原文本拆分为句子，在每个句子中过滤掉停用词（可选），并只保留指定词性的单词（可选）。由此可以得到句子的集合和单词的集合。TextRank算法是利用局部词汇之间关系（共现窗口）对后续关键词进行排序，直接从文本本身抽取。其主要步骤如下：

1、把给定的文本T按照完整句子进行分割，

2、对于每个句子，进行分词和词性标注处理，并过滤掉停用词，只保留指定词性的单词，如名词、动词、形容词

3、构建候选关键词图G = (V,E)，其中V为节点集，由（2）生成的候选关键词组成，然后采用共现关系（co-occurrence）构造任两点之间的边，两个节点之间存在边仅当它们对应的词汇在长度为K的窗口中共现，K表示窗口大小，即最多共现K个单词。

4、根据上面公式，迭代传播各节点的权重，直至收敛。

5、对节点权重进行倒序排序，从而得到最重要的T个单词，作为候选关键词。

6、由5得到最重要的T个单词，在原始文本中进行标记，若形成相邻词组，则组合成多词关键词。

### 使用TextRank提取关键短语
参照“使用TextRank提取关键词”提取出若干关键词。若原文本中存在若干个关键词相邻的情况，那么这些关键词可以构成一个关键短语。

例如，在一篇介绍“支持向量机”的文章中，可以找到三个关键词 支持、向量、机，通过关键短语提取，可以得到

支持向量机.